In [1]:
import gensim
import smart_open
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
from gensim.models.callbacks import CallbackAny2Vec

import nltk
from nltk.corpus import stopwords

In [2]:
import gc
gc.collect()

0

In [3]:
cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

In [4]:
cores
stop_words = nltk.corpus.stopwords.words('english')

In [5]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        loop = 0
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                word_list = nltk.tokenize.word_tokenize(line)
                word_list2 = [w.strip() for w in word_list if w.strip() not in stop_words]
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(" ".join(word_list2)), [i])

            loop += 1
            if loop % 10000 == 0:
                print(loop)

            if loop == 1000000:
                break

In [6]:
documents = list(read_corpus('./data/dataTitle.txt'))

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000


In [11]:
len(documents)

1000000

In [12]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0
        self.batch = 1

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

    def on_train_begin(self, model):
        print("train started")
        self.batch = 1

    def on_train_end(self, model):
        print("train ended")

In [13]:
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=2, epochs=10, workers=cores),
    # PV-DM w/average
#     Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=2, epochs =10, workers=cores),
]

In [14]:
models[0].build_vocab(documents)
print(str(models[0]))
# models[1].reset_from(models[0])
# print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,mc2,s0.001,t16)


In [32]:
for index, model in enumerate(models):
    %time model.train(documents, total_examples=model.corpus_count, epochs=10, callbacks=[EpochLogger()])
    model.save('./data/DbowModel' + str(index) + ".model")

train started
Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
train ended
CPU times: user 1h 11min 46s, sys: 17min 4s, total: 1h 28min 50s
Wall time: 32min 32s


In [15]:
for index, model in enumerate(models):
    models[index] = Doc2Vec.load('./data/DbowModel' + str(index) + ".model")

In [102]:
from pprint import pprint
def get(model, st):
    tokens = st.lower().split()
    new_vector = model.infer_vector(tokens)
    sims = model.docvecs.most_similar([new_vector], topn=20)
    for index, val in sims:
        print(str(' '.join(documents[index].words)) + " - " + str(val))

In [116]:
guess_it = "Travelpro Luggage Crew 11 21 Carry-on Expandable Spinner w/Suiter and USB Port, Black" 
for model in models:
    print(str(model))
    get(model, guess_it)
    print("\n\n")
    
# Have to use dbow

Doc2Vec(dbow+w,d200,n5,w8,mc2,s0.001,t16)
ninewest expandable spinner luggage - 0.78464674949646
nautica hardside expandable spinner luggage - 0.7668018341064453
swissgear baden expandable spinner suitcase silver - 0.7467473149299622
swissgear meyrin expandable spinner suitcase dark blue - 0.7464632391929626
swissgear travel gear expandable hardside spinner luggage - 0.7431485652923584
swissgear baden expandable spinner suitcase silver - 0.7429184317588806
avolve expandable spinner black one size - 0.7416092157363892
steve madden luggage expandable softside suitcase with spinner wheels - 0.7402724623680115
avolve expandable spinner black one size - 0.7398602366447449
nautica carry on expandable spinner luggage - 0.7389228940010071
luggage quilte lightweight expandable spinner patriot blue - 0.7364516258239746
nautica carry on expandable spinner luggage - 0.7327922582626343
luggage duotone wheel luggage spinner - 0.7274551391601562
kemyer lightweight pc expandable hardside spinner lugga

In [18]:
models[0].docvecs.save_word2vec_format('./data/dbow.txt', binary=False)

In [19]:
import numpy as np

In [20]:
EMBEDDING_FILE = "./data/dbow.txt"
DIMENSION_FILE = "./data/dataDimesions.txt"

In [89]:
X = []
with open(EMBEDDING_FILE, "r") as ip:
    flg = 0
    
    for line in ip:
        if flg:
            data = line.split()
            del data[0]
            data = [float(i) for i in data]
#             npd = np.array(data, dtype=float)
#             print(npd)
            X.append(data)
#             X = np.concatenate(X, npd)
            
            if flg == 1000000:
                break
        flg += 1

In [90]:
# print(X)
# np.asarray(X).shape
# X.shape

In [91]:
Y = []
with open(DIMENSION_FILE, "r") as op:
    for line in op:
        data = line.split()
        data = list(map(float, data))
        Y.append(data)
        if len(Y) == len(X):
            break

In [92]:
print(len(X), len(Y))
print(len(X[0]), len(Y[0]))

1000000 1000000
200 4


In [86]:
import numpy as np

# print(X[:2])
# print(len(X), len(X[0]))
# print(len(Y), len(Y[0]))
# docs = np.array(X[i] for i in X)
docs = np.array([i for i in X])
target = np.array(Y)

# print(docs[:2])
# print(target[:2])
# docs.reshape([1100001, 200])
print(docs.shape, target.shape)
ratio = int( docs.shape[0]/10 )
X_train = docs[ratio:]
X_test = docs[:ratio]
y_train = target[ratio:,:]
y_test = target[:ratio,:]

(100000, 200) (100000, 4)


In [ ]:
X_train[:2]

In [98]:
doc_embedding_size = 200
generations = 10000

# Neural Network


In [99]:
from tensorflow import keras

model = keras.Sequential()
activation_function = keras.layers.PReLU()

# Input Layer
model.add(keras.layers.Dense(405, kernel_initializer=keras.initializers.random_uniform, input_dim=doc_embedding_size,
                             kernel_regularizer=keras.regularizers.l2(0.01),
                             bias_regularizer=keras.regularizers.l2(0.01), name="input"))
model.add(activation_function)

# Hidden Layer
model.add(keras.layers.Dense(405, kernel_initializer='normal', name="hidden1"))
model.add(activation_function)
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(405, kernel_initializer='normal', name="hidden2"))
model.add(activation_function)
model.add(keras.layers.Dropout(0.2))

# Output Layer
model.add(keras.layers.Dense(4, kernel_initializer='normal', name="output", activation="relu"))

model.compile(loss='mean_absolute_error', optimizer="Adam", metrics=['mean_absolute_error'])

checkpoint = keras.callbacks.ModelCheckpoint("./temp/best.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                                 mode='auto')

earlyStopping = keras.callbacks.EarlyStopping(patience=16, mode="auto")
callbacks_list = [earlyStopping]

# model = create_neural_model()
model.fit(X_train, y_train, epochs=generations, batch_size=256, validation_split=0.1, callbacks=callbacks_list)
print("Current one: ", model.evaluate(X_test, y_test, batch_size=256))

Train on 81000 samples, validate on 9000 samples
Epoch 1/10000
81000/81000 [==============================] - 4s 50us/step - loss: 311.7402 - mean_absolute_error: 310.6904 - val_loss: 231.6713 - val_mean_absolute_error: 230.8048
Epoch 2/10000
81000/81000 [==============================] - 1s 18us/step - loss: 255.1163 - mean_absolute_error: 254.1193 - val_loss: 212.7802 - val_mean_absolute_error: 211.6787
Epoch 3/10000
81000/81000 [==============================] - 1s 18us/step - loss: 246.9152 - mean_absolute_error: 245.7516 - val_loss: 209.6385 - val_mean_absolute_error: 208.4286
Epoch 4/10000
81000/81000 [==============================] - 1s 18us/step - loss: 243.9210 - mean_absolute_error: 242.6744 - val_loss: 210.0077 - val_mean_absolute_error: 208.7315
Epoch 5/10000
81000/81000 [==============================] - 1s 18us/step - loss: 241.6919 - mean_absolute_error: 240.3890 - val_loss: 211.9230 - val_mean_absolute_error: 210.5900
Epoch 6/10000
81000/81000 [========================